# Read arrow file in R

In [1]:
ncvote <- arrow::read_ipc_file("ncvoter_Statewide.arrow")  # as a tibble
tibble::glimpse(ncvote)

Rows: 8,778,585
Columns: 67
$ county_id                <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1…
$ county_desc              <fct> ALAMANCE, ALAMANCE, ALAMANCE, ALAMANCE, ALAMANCE, ALAMANCE, ALAMANCE, ALAMANCE, ALAMANCE, ALAMANCE, …
$ voter_reg_num            <int> 9005990, 9178574, 9205561, 9048723, 9019674, 9129589, 9194595, 9041748, 9144384, 9144385, 9021947, 9…
$ ncid                     <chr> "AA56273", "AA201627", "AA216996", "AA98377", "AA69747", "AA170513", "BM49306", "AA91549", "AA125250…
$ last_name                <fct> AABEL, AARDEN, AARMSTRONG, AARON, AARON, AARON, AARON, AARON, AARON, AARON, AARON, AARONSON, AARONSO…
$ first_name               <fct> RUTH, JONI, TIMOTHY, CHRISTINA, CLAUDIA, JAMES, KIMBERLY, NATHAN, RICHARD, SANDRA, WILLIE, GENA, MIC…
$ middle_name              <fct> EVELYN, AUTUMN, DUANE, CASTAGNA, HAYDEN, MICHAEL, GREEN, EDWARD, BRIAN, ESCOBAR, DALE, HOLT, CHARLES…
$ name_suffix_lbl          

There were 20 warnings (use warnings() to see them)


Most of the warnings relate to coding of special characters.  I still haven't worked out what the coding in the original tab-separated-file is.

Counting the voter status here is similar to the Julia form

In [2]:
dplyr::count(ncvote, status_cd, voter_status_desc, sort=TRUE)

# A tibble: 5 × 3
  status_cd voter_status_desc       n
  <fct>     <fct>               <int>
1 A         ACTIVE            7089338
2 R         REMOVED           1100070
3 I         INACTIVE           401159
4 D         DENIED             173591
5 S         TEMPORARY           14427

Reading a large Arrow IPC file as a `data.frame` is slow because the data must be copied to a different form.  R only allows for one type of integer (32-bit unsigned), one type of floating point representation and factors must have string representations.

Also, R's representation of missing values, via sentinels, is different from the Arrow representation (optional bitvector) so you can't memory-map and just pass pointers.

You can avoid the translation/copying and work with a pyarrow-like representation.

In [3]:
tbl = arrow::read_ipc_file("ncvoter_Statewide.arrow", as_data_frame = FALSE)
tbl

Table
8778585 rows x 67 columns
$county_id <int8 not null>
$county_desc <dictionary<values=string, indices=int8> not null>
$voter_reg_num <int32 not null>
$ncid <string not null>
$last_name <dictionary<values=string, indices=int32> not null>
$first_name <dictionary<values=string, indices=int32>>
$middle_name <dictionary<values=string, indices=int32>>
$name_suffix_lbl <dictionary<values=string, indices=int8>>
$status_cd <dictionary<values=string, indices=int8> not null>
$voter_status_desc <dictionary<values=string, indices=int8> not null>
$reason_cd <dictionary<values=string, indices=int8>>
$voter_status_reason_desc <dictionary<values=string, indices=int8>>
$res_street_address <string not null>
$res_city_desc <dictionary<values=string, indices=int16>>
$state_cd <dictionary<values=string, indices=int8>>
$zip_code <int32>
$mail_addr1 <string>
$mail_addr2 <dictionary<values=string, indices=int32>>
$mail_addr3 <dictionary<values=string, indices=int16>>
$mail_addr4 <dictionary<values=string,

In [4]:
class(tbl)

[1] "Table"        "ArrowTabular" "ArrowObject"  "R6"          

Many of the dplyr functions have methods for this type of table.  Usually you would apply `as.data.frame` to the result before printing.

In [5]:
as.data.frame(dplyr::count(tbl, status_cd, voter_status_desc, sort=TRUE))

  status_cd voter_status_desc       n
1         A            ACTIVE 7089338
2         R           REMOVED 1100070
3         I          INACTIVE  401159
4         D            DENIED  173591
5         S         TEMPORARY   14427